In [1]:
!pip install --upgrade tensorflow jax jaxlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 6.0 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 MB 16.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 54.5 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 60.3 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.5.1
    Uninstalling jaxlib-0.5.1:
      Successfully uninstalled jaxlib-0.5.1
  Attempting uninstall: tensorflow
    Found existing installation: te

In [2]:
import os
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Dropout, Conv2D, MaxPooling2D, Flatten, Activation
# from keras.layers.wrappers import TimeDistributed
from keras.layers import TimeDistributed

2025-08-01 09:33:30.564040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754040810.586332      36 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754040810.593001      36 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754040810.610873      36 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754040810.610906      36 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754040810.610908      36 computation_placer.cc:177] computation placer alr

In [3]:
from pathlib import Path

def count_png_files(directory):
    return sum(1 for file in Path(directory).rglob('*.png'))

path = '/kaggle/input/cwt-images-eeg-fatigue-data'
num_samples = count_png_files(path)
print("Total .png files:", num_samples)

Total .png files: 12473


In [4]:
import cv2
import numpy as np
from tqdm import tqdm

# image_shape = (224, 224, 3)  # adjust as needed
image_shape = (224, 224, 3)

# Preallocate empty array
X = np.zeros((num_samples, *image_shape), dtype=np.float32)  # or float32 if needed
Y = np.zeros((num_samples,), dtype=np.float32)

index = 0

for sub_id in tqdm(sorted(os.listdir(path + "/Images"))):
    for label in sorted(os.listdir(path + "/Images/" + sub_id)):
        for image in sorted(os.listdir(path + "/Images/" + sub_id + "/" + label)):
            image_path = os.path.join(path, "Images", sub_id, label, image)
            image_array = cv2.imread(image_path)
            image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
            #image_array = cv2.resize(image_array, (128,128))

            if image_array is None:
                print("skipping")
                continue  # skip corrupted or unreadable images

            # X.append(image_array / 255.0)
            X[index] = image_array / 255.0

            if label.lower() == "fatigue":
                Y[index] = 1
            elif label.lower() == "normal":
                Y[index] = 0

            index += 1

100%|██████████| 12/12 [01:28<00:00,  7.37s/it]


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(7,7))
# plt.imshow(Normal_npy[0][0])
plt.imshow(X[0])
plt.gca().xaxis.set_major_locator(plt.NullLocator())
plt.gca().yaxis.set_major_locator(plt.NullLocator())

In [6]:
from sklearn.model_selection import train_test_split
# Splitting the dataset into the Training set and Test set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 0)

len(x_train), len(x_test)

(8731, 3742)

In [7]:
y_test[:100]

array([0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
       1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0.],
      dtype=float32)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define input shape
input_shape = (224, 224, 3)

# Build a CNN model
model = Sequential()

# Convolutional Layer
model.add(Conv2D(32, (7, 7), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Layer
model.add(Conv2D(32, (7, 7), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and Dense Layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # Use 'sigmoid' for binary output

# Summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1754040928.328517      36 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 218, 218, 32)   │         4,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 109, 109, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 103, 103, 32)   │        50,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 51, 51, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 83232)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     5,326,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,381,986 (20.53 MB)

 Trainable params: 5,381,986 (20.53 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model

from tensorflow.keras.utils import plot_model

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100


I0000 00:00:1754040949.146734     110 service.cc:152] XLA service 0x7a5a6800a8e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754040949.146785     110 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1754040949.481503     110 cuda_dnn.cc:529] Loaded cuDNN version 90300


  5/273 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.5734 - loss: 1.7250

I0000 00:00:1754040954.909709     110 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


273/273 ━━━━━━━━━━━━━━━━━━━━ 30s 84ms/step - accuracy: 0.5283 - loss: 0.7665 - val_accuracy: 0.5080 - val_loss: 0.6931
Epoch 2/100
273/273 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.5633 - loss: 0.6521 - val_accuracy: 0.8592 - val_loss: 0.3365
Epoch 3/100
273/273 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.8635 - loss: 0.3179 - val_accuracy: 0.9228 - val_loss: 0.1912
Epoch 4/100
273/273 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.9153 - loss: 0.2106 - val_accuracy: 0.9468 - val_loss: 0.1421
Epoch 5/100
273/273 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.9380 - loss: 0.1500 - val_accuracy: 0.9514 - val_loss: 0.1262
Epoch 6/100
273/273 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.9501 - loss: 0.1263 - val_accuracy: 0.9540 - val_loss: 0.1237
Epoch 7/100
273/273 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.9598 - loss: 0.1020 - val_accuracy: 0.9631 - val_loss: 0.1132
Epoch 8/100
273/273 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.9677 - loss: 0.0871 - val_